In [7]:
pip install gurobipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [16]:
import gurobipy as gp
import pandas as pd

C:\Users\20182366\AppData\Local\Temp\ipykernel_13492\3690940260.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
def get_num_districts(state):

    return None

In [14]:
state = 'RI'
filename = "districting_2020/{}/counties/graph/{}.population".format(state, state)


with open('Numberofdistricts.txt', 'r') as file:
    for line in file:
        parts = line.strip().split('\t')
        if parts[0] == state:
            num_districts =  int(parts[1])

with open(filename, "r") as file:
    # Skip the first line
    next(file)
    # Count the number of lines, each line represents a county
    num_counties = sum(1 for line in file)

print("Number of districts: ", num_districts)
print("Number of counties: ", num_counties)


Number of districts:  2
Number of counties:  5


In [13]:
# Model
m = gp.Model("district")

state = 'RI'
adjacency_matrix_file = r'districting_2020\2020\RI\counties\graph\RI_distances.csv'

# Set decision variable x for each node j wether it is in district k
for i in range(num_counties):
    for j in range(num_districts):
        x = m.addVar(vtype=gp.GRB.BINARY, name="x_{}_{}".format(i, j))

# Set decision variable y for each pair of nodes i and k, and each district j
for i in range(num_counties):
    for k in range(num_counties):
        for j in range(num_districts):
            y = m.addVar(vtype=gp.GRB.BINARY, name="y_{}_{}_{}".format(i, k, j))







Restricted license - for non-production use only - expires 2025-11-24


In [29]:
distance = r'districting_2020\RI\counties\graph\RI_distances.csv'
distance_df = pd.read_csv(adjacency_matrix_file, header= None, skiprows=1)
distance_df = distance_df.iloc[:, 1:]

In [30]:
distance_df

,1,2,3,4,5
0,0,31,48,22,53
1,31,0,23,25,44
2,48,23,0,31,31
3,22,25,31,0,31
4,53,44,31,31,0
